In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

# Load the data
data = pd.read_csv('testTimeSeriesData.csv', usecols=[1])
data = np.array(data).astype('float32')

# Define the sequence length and batch size
sequence_length = 24
batch_size = 32

# Create sequences of data
def create_sequences(data, sequence_length, batch_size):
    num_sequences = len(data) - sequence_length
    num_batches = num_sequences // batch_size
    data = data[:num_batches * batch_size + sequence_length]
    x = np.zeros((num_batches, batch_size, sequence_length))
    y = np.zeros((num_batches, batch_size, 1))
    for i in range(num_batches):
        x[i] = data[i * batch_size:i * batch_size + sequence_length].reshape(batch_size, sequence_length)
        y[i] = data[i * batch_size + sequence_length:(i + 1) * batch_size + sequence_length].reshape(batch_size, 1)
    return x, y

# Create the sequences
x, y = create_sequences(data, sequence_length, batch_size)

# Define the model
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(Model, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(self.rnn.num_layers, x.size(0), self.rnn.hidden_size).to(device)
        c0 = torch.zeros(self.rnn.num_layers, x.size(0), self.rnn.hidden_size).to(device)
        out, _ = self.rnn(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the hyperparameters
input_size = 1
hidden_size = 64
num_layers = 2
output_size = 1
learning_rate = 0.001
num_epochs = 50

# Instantiate the model
model = Model(input_size, hidden_size, num_layers, output_size).to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for i in range(x.shape[0]):
        inputs = torch.from_numpy(x[i]).to(device)
        targets = torch.from_numpy(y[i]).to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Test the model
with torch.no_grad():
    inputs = torch.from_numpy(x[0]).to(device)
    outputs = model(inputs)
    print('Predicted traffic volume:', outputs.cpu().numpy()[0][0])


NameError: name 'loss' is not defined